In [1]:
import os
import pickle
import gzip
import pandas as pd
import numpy as np
import torch
from time import time
from tqdm import tqdm

from src.datasets.caltech256 import Caltech256DataModule
from src.featuring.rgb_histogram import RGBHistogram
from src.storage.VectorDBStore import VectorDBStore
from src.retrieval.KNN import KNNRetrieval
from src.pipeline import CBIR
from src.metrics import average_precision, recall, hit_rate


In [2]:
os.makedirs('out', exist_ok=True)

# INDEXING

In [20]:
TRAIN_SIZE = 24607  # Số lượng ảnh để index
TEST_SIZE = 1000   # Giảm test size để test nhanh hơn với nhiều combinations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo dataset
import os
root_path = os.path.abspath('data/caltech-256/256_ObjectCategories')
print(f"🔍 Dataset root: {root_path}")
print(f"🔍 Exists: {os.path.exists(root_path)}")
data_module = Caltech256DataModule(batch_size=32, root=root_path)
data_module.setup()
train_loader = data_module.train_dataloader()
test_loader = data_module.test_dataloader()

# Tất cả combinations để test
n_bins = [4, 8, 12]
h_types = ["global", "region"]
metrics = ["cosine", "euclidean"]

print(f"🧪 Testing {len(n_bins)} bins × {len(h_types)} h_types × {len(metrics)} metrics = {len(n_bins)*len(h_types)*len(metrics)} combinations")

# Hàm để test một combination
def test_combination(n_bin, h_type, metric):
    config_name = f"{n_bin}bin_{h_type}_{metric}"
    print(f"\n🔬 Testing: n_bin={n_bin}, h_type={h_type}, metric={metric}")
    
    # Khởi tạo CBIR pipeline 
    feature_extractor = RGBHistogram(n_bin=n_bin, h_type=h_type)
    retrieval = KNNRetrieval(metric=metric)
    storage = VectorDBStore(retrieval)
    cbir = CBIR(feature_extractor, storage)
    
    # Indexing
    print(f"Indexing {TRAIN_SIZE} images...")
    start = time()
    indexed = 0
    
    for images, labels, _ in tqdm(train_loader, desc=f"Indexing ({config_name})"):
        if indexed >= TRAIN_SIZE:
            break
    
        if device.type == "cuda":
            images = images.to(device)
    
        count = min(len(images), TRAIN_SIZE - indexed)
        images = images[:count]
        images = (images.cpu().numpy().transpose(0, 2, 3, 1) * 255).astype(np.uint8)
    
        cbir.add_images(images)
        indexed += count
    
    indexing_time = time() - start
    print(f"Indexed {indexed} images in {indexing_time:.2f}s")
    
    # Save model
    model_path = f'out/caltech256_model_{config_name}.pkl.gz'
    with gzip.open(model_path, 'wb') as f:
        pickle.dump(cbir, f)
    
    file_size = os.path.getsize(model_path) / 1024 / 1024
    print(f"💾 Model saved: {file_size:.2f} MB")
    
    return cbir, indexing_time, file_size, indexed, config_name

# Test tất cả combinations
results_comparison = {}
models = {}
all_configs = []

for n_bin in n_bins:
    for h_type in h_types:
        for metric in metrics:
            config_name = f"{n_bin}bin_{h_type}_{metric}"
            all_configs.append((n_bin, h_type, metric, config_name))
            
            cbir, indexing_time, file_size, indexed, _ = test_combination(n_bin, h_type, metric)
            
            models[config_name] = cbir
            results_comparison[config_name] = {
                'n_bin': n_bin,
                'h_type': h_type, 
                'metric': metric,
                'indexing_time': indexing_time,
                'file_size': file_size,
                'indexed_images': indexed
            }

print(f"\n✅ Completed indexing for all {len(all_configs)} combinations!")

🔍 Dataset root: D:\AI\Food-CBIR\src\evaluation\data\caltech-256\256_ObjectCategories
🔍 Exists: True
📂 Found 256 valid categories
📊 Loaded 29780 total images from 256 classes
📋 Train: 23824 images
📂 Found 256 valid categories
📊 Loaded 29780 total images from 256 classes
📋 Test: 5956 images
Train: 23824, Test: 5956
🧪 Testing 3 bins × 2 h_types × 2 metrics = 12 combinations

🔬 Testing: n_bin=4, h_type=global, metric=cosine
Indexing 24607 images...


Indexing (4bin_global_cosine): 100%|██████████| 745/745 [04:09<00:00,  2.98it/s]


Indexed 23824 images in 249.59s
💾 Model saved: 6.14 MB

🔬 Testing: n_bin=4, h_type=global, metric=euclidean
Indexing 24607 images...


Indexing (4bin_global_euclidean): 100%|██████████| 745/745 [02:23<00:00,  5.19it/s]


Indexed 23824 images in 143.44s
💾 Model saved: 6.14 MB

🔬 Testing: n_bin=4, h_type=region, metric=cosine
Indexing 24607 images...


...

Indexed 23824 images in 380.95s
💾 Model saved: 77.96 MB

🔬 Testing: n_bin=12, h_type=region, metric=cosine
Indexing 24607 images...


Indexing (12bin_region_cosine): 100%|██████████| 745/745 [12:58<00:00,  1.04s/it]


Indexed 23824 images in 778.45s
💾 Model saved: 342.13 MB

🔬 Testing: n_bin=12, h_type=region, metric=euclidean
Indexing 24607 images...


Indexing (12bin_region_euclidean): 100%|██████████| 745/745 [13:07<00:00,  1.06s/it]


Indexed 23824 images in 787.76s
💾 Model saved: 342.10 MB

✅ Completed indexing for all 12 combinations!


# Evaluation

In [21]:
# Hàm để evaluate một model
def evaluate_model(cbir, metric_name):
    print(f"\n📊 Evaluating {metric_name.upper()} model...")
    start = time()
    results = []
    ground_truth = []
    tested = 0

    # Get dataset targets for evaluation
    dataset_targets = []
    for images, labels, _ in train_loader:
        if len(dataset_targets) >= indexed:
            break
        count = min(len(labels), indexed - len(dataset_targets))
        dataset_targets.extend(labels[:count].numpy())
    dataset_targets = np.array(dataset_targets)

    # Query với k=10 để tính mAP@1,5,10
    MAX_K = 10

    for images, labels, _ in tqdm(test_loader, desc=f"Testing ({config_name})"):
        if tested >= TEST_SIZE:
            break

        if device.type == "cuda":
            images = images.to(device)

        count = min(len(images), TEST_SIZE - tested)
        images = images[:count]
        labels = labels[:count]

        images = (images.cpu().numpy().transpose(0, 2, 3, 1) * 255).astype(np.uint8)

        for image in images:
            if tested >= TEST_SIZE:
                break
            # Query với k=10 để tính mAP@1,5,10
            result = cbir.query_similar_images(image, k=MAX_K)
            results.append(result)
            tested += 1

        ground_truth.extend(labels.numpy())

    retrieval_time = time() - start
    print(f"Tested {tested} images in {retrieval_time:.2f}s")

    # Calculate metrics cho k=1, k=5, k=10
    k_values = [1, 5, 10]
    metrics_data = {}

    print(f"📈 Calculating metrics for k={k_values}...")

    for k in k_values:
        map_k, recall_k, hit_k = [], [], []

        for r, gt in zip(results, ground_truth):
            # Lấy top-k results
            top_k_results = r[:k]
            indices = [item.index for item in top_k_results]
            preds = np.take(dataset_targets, indices)
            relevant = np.where(dataset_targets == gt)[0]

            map_k.append(average_precision(preds.tolist(), [gt], k))
            recall_k.append(recall(indices, relevant, k))
            hit_k.append(hit_rate(preds.tolist(), [gt], k))

        # Store metrics
        metrics_data[f'mAP@{k}'] = np.mean(map_k)
        metrics_data[f'Recall@{k}'] = np.mean(recall_k)
        metrics_data[f'HitRate@{k}'] = np.mean(hit_k)

        print(f"   k={k}: mAP={np.mean(map_k):.4f}, Recall={np.mean(recall_k):.4f}, HR={np.mean(hit_k):.4f}")

    # Tính average mAP score cho ranking
    avg_map = (metrics_data['mAP@1'] + metrics_data['mAP@5'] + metrics_data['mAP@10']) / 3
    metrics_data['avg_mAP'] = avg_map

    # Add config và timing metrics
    metrics_data.update(results_comparison[config_name])
    metrics_data['retrieval_time'] = retrieval_time
    metrics_data['tested_images'] = tested
    metrics_data['config_name'] = config_name

    return metrics_data

# Evaluate tất cả 12 models
print(f"\n🚀 Starting evaluation of all {len(models)} models...")
all_results = []

for config_name in models.keys():
    cbir = models[config_name]
    metrics = evaluate_model(cbir, config_name)
    all_results.append(metrics)
    results_comparison[config_name].update(metrics)

# Tìm best model dựa trên average mAP@1,5,10
print(f"\n🔍 Finding best model based on average mAP@1,5,10...")
best_config = max(results_comparison.keys(), key=lambda x: results_comparison[x]['avg_mAP'])
best_model = models[best_config]
best_metrics = results_comparison[best_config]

print(f"\n🏆 BEST MODEL: {best_config}")
print(f"   Average mAP: {best_metrics['avg_mAP']:.4f}")
print(f"   mAP@1: {best_metrics['mAP@1']:.4f}")
print(f"   mAP@5: {best_metrics['mAP@5']:.4f}")
print(f"   mAP@10: {best_metrics['mAP@10']:.4f}")

# Save best model
best_model_path = 'out/best_color.pkl.gz'
with gzip.open(best_model_path, 'wb') as f:
    pickle.dump(best_model, f)
best_file_size = os.path.getsize(best_model_path) / 1024 / 1024
print(f"💾 Best model saved as: best_color.pkl.gz ({best_file_size:.2f} MB)")

# Save config của best model
best_config_info = {
    'config_name': best_config,
    'n_bin': best_metrics['n_bin'],
    'h_type': best_metrics['h_type'],
    'metric': best_metrics['metric'],
    'avg_mAP': best_metrics['avg_mAP'],
    'mAP@1': best_metrics['mAP@1'],
    'mAP@5': best_metrics['mAP@5'],
    'mAP@10': best_metrics['mAP@10']
}

import json
with open('out/best_color_config.json', 'w') as f:
    json.dump(best_config_info, f, indent=2)
print("📋 Best model config saved as: best_color_config.json")

# So sánh tất cả kết quả
print(f"\n📊 ALL RESULTS RANKING (by avg mAP)")
print("=" * 80)
print(f"{'Rank':<4} {'Config':<20} {'AvgmAP':<8} {'mAP@1':<8} {'mAP@5':<8} {'mAP@10':<8}")
print("-" * 80)

# Sort by avg_mAP descending
sorted_configs = sorted(results_comparison.items(), key=lambda x: x[1]['avg_mAP'], reverse=True)

for rank, (config, data) in enumerate(sorted_configs, 1):
    print(f"{rank:<4} {config:<20} {data['avg_mAP']:<8.4f} {data['mAP@1']:<8.4f} {data['mAP@5']:<8.4f} {data['mAP@10']:<8.4f}")

# Save detailed results
results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values('avg_mAP', ascending=False)
results_df.to_csv('out/all_color_combinations_results.csv', index=False)
print(f"\n✅ Detailed results saved to: all_color_combinations_results.csv")

# Top 3 summary
print(f"\n🥇 TOP 3 CONFIGURATIONS:")
for i, (config, data) in enumerate(sorted_configs[:3], 1):
    emoji = ["🥇", "🥈", "🥉"][i-1]
    print(f"{emoji} {config}: avg_mAP={data['avg_mAP']:.4f} (n_bin={data['n_bin']}, h_type={data['h_type']}, metric={data['metric']})")

if device.type == "cuda":
    torch.cuda.empty_cache()


🚀 Starting evaluation of all 12 models...

📊 Evaluating 4BIN_GLOBAL_COSINE model...


Testing (4bin_global_cosine):  13%|█▎        | 25/187 [01:04<07:00,  2.59s/it]


KeyboardInterrupt: 